<a href="https://colab.research.google.com/github/Reptilefury/coursera-machine-learning/blob/main/Pytorch_LFW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
from torchvision.transforms import ToTensor,Resize 
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:

#Load the data
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

#Unzip the data 
!tar -xvf /content/lfw.tgz


Load the data from Image Folder

In [3]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(224),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ToTensor()
])

DATA_DIR = "/content/lfw"
train_ds = ImageFolder(root =DATA_DIR,transform= TRANSFORM_IMG)
test_ds = ImageFolder(root=DATA_DIR,transform=TRANSFORM_IMG)

#DataLoader 
train_loader = DataLoader(train_ds,batch_size=64,shuffle=True)
test_loader = DataLoader(test_ds,batch_size=64,shuffle=True)


In [17]:
num_classes = len(train_ds.classes)
num_classes

5749

Use ray tune  a hyperparameter tuning library

In [ ]:
!pip install -U "ray[default]"

In [11]:
from functools import partial 
import numpy as np
import os 
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision 
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


def load_data(data_dir="./"):
  transform = transforms.Compose([
      transforms.Resize(224),
      transforms.ToTensor(),
      transforms.Normalize(( 0.5,0.5,0.5),(0.5,0.5,0.5))
  ])
  train_data = torchvision.datasets.CIFAR10(root=data_dir,train=True,transform=transform,download=True)
  test_data = torchvision.datasets.CIFAR10(root=data_dir,train=True,transform=transform,download=True)
  return train_data,test_data
  

Define the Neural Network

In [20]:
class NeuralNet(nn.Module):
  def __init__(self,l1=120,l2=84):
    super(NeuralNet,self).__init__()
    #First convolution block
    self.conv1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3,padding=1)
    self.conv2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1)
    self.conv3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1)
    
    #Second convolution block
    self.conv4 = nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1)
    self.conv5 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1)    
    
    #Third convolution block
    self.conv6 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1)
    self.conv7 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1)
    self.conv8 = nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=1)
    
    #Fourth convolution block 
    self.conv9 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1)
    self.conv10 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1)
    self.conv11 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1)
    self.conv12 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1)
    self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
    self.conv13 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,padding=1)

    #Dense layer
    self.fc1 = nn.Linear(512 * 3 * 3,l1)
    self.fc2 = nn.Linear(l1,l2)
    self.fc3 = nn.Linear(l2,num_classes)
  
  def call(self,x):

    #1st convolution block   
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = self.maxpool(x)
    
    #2nd convolution block
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = self.maxpool(x)
   
    #3rd convolution block
    x = F.relu(self.conv5(x))
    x = F.relu(self.conv6(x))
    x = F.relu(self.conv7(x))
    x = F.maxpool(x)
    
    #4th convolution block
    x = F.relu(self.conv8(x))
    x = F.relu(self.conv9(x))
    x = F.relu(self.conv10(x))
    x = self.maxpool(x)
    
    #5th  convolution block
    x = F.relu(self.conv11(x))
    x = F.relu(self.conv12(x))
    x = F.relu(self.conv13(x))
    x = self.maxpool(x)
    
    #6th convolution block 
    x = x.view(-1,512 * 3 * 3)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    
    return x 
